In [1]:
import sys
sys.path.append('./../../')

import torch
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
from user_fun.pde import diff

np.random.seed(1)
torch.manual_seed(1)


from user_fun.solver.cp_solver import CloudPointSolver
from user_fun.geom import line_linspace,generate_points_in_rectangle

a = 0.02
density = 32
init_input = line_linspace([0,0],[1,0],density*2)
init_output = np.sin(2*np.pi *init_input[:,[0]])

left_input = line_linspace([0,0],[0,3],density*3)
left_output = np.sin(np.pi *left_input[:,[1]])
right_input = line_linspace([1,0],[1,3],density*3)
right_output = np.sin(np.pi *right_input[:,[1]])

field_input = generate_points_in_rectangle([0,0],[1,3],density*density*3)
field_output = np.zeros((field_input.shape[0],1))

%matplotlib widget
from visual import process_point_sets,visualize_point_sets_interactive

# 示例数据
point_sets = {
    "left bound": [left_input, left_output],
    "right bound": [right_input, right_output],
    "init condition": [init_input, init_output],
    "residual points": [field_input, field_output]
}


results = process_point_sets(point_sets)
visualize_point_sets_interactive(results,xlim = [-0.2,1.2],ylim = [0,3],zlim = [-1,1])


Output()

In [3]:
from user_fun import bc
from torch import nn
loss_fn = nn.MSELoss()
left_loss = bc.data_loss_factory(loss_fn)
right_loss = bc.data_loss_factory(loss_fn)
init_loss = bc.data_loss_factory(loss_fn)


def heat_loss(model, data):
    input,output = data
    input.requires_grad=True
    

    # 数据提取
    x = input[:,[0]]
    t = input[:,[1]]
    use_input = torch.cat([x,t],dim = 1)
    U = model(use_input)
    u = U[:,[0]]

    # 计算一阶导
    dudx = diff(u, x)
    dudt = diff(u, t)

    # 计算二阶导
    du2dx2 = diff(dudx, x)

    loss = dudt - 0.02 * du2dx2
    loss = loss_fn(loss, output)
    return loss

cp_list = [
    [left_input, left_output],
    [right_input, right_output],
    [init_input, init_output],
    [field_input, field_output]
]

loss_list = [
    left_loss, right_loss, init_loss, heat_loss
]

solver = CloudPointSolver(
    cp_list,
    loss_list,
    model = [2, 50, 50, 50, 1],
    optimizer = "adam")


epoch_num = 900
error_list = []
for i in range(epoch_num):
    solver.train_step()
    if i%30 == 0:
        print(f'-------------{i}----------')
        solver.test_step(print_flag=True)
        relative_error = eval(solver)
        error_list.append(relative_error)
        print(f'relative error is {relative_error}')



y_cpu = solver.model_eval(field_input)


-------------0----------
loss is [5.3971338e-01 5.7081437e-01 4.8268154e-01 3.6908179e-05 1.5932462e+00]


TypeError: eval() arg 1 must be a string, bytes or code object

In [ ]:
from user_fun.io import read_comsol
from user_fun.io.read_comsol import process_comsol_time_table
tb = read_comsol.comsol_read('time_heat.csv', skiprows=8)
tb = process_comsol_time_table(tb, space_dimension=1)
def eval(solver):
    x = tb['x'].to_numpy()
    t = tb['t'].to_numpy()
    u = tb['u (K)'].to_numpy()
    plot_input = np.vstack([x, t]).T
    
    pred_u = solver.model_eval(plot_input)
    return np.mean(np.abs(pred_u[:,0] - u))
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.tri as tri

print(tb.head(8))

x = tb['x'].to_numpy()
t = tb['t'].to_numpy()
u = tb['u (K)'].to_numpy()
plot_input = np.vstack([x, t]).T
plot_output = solver.model_eval(plot_input)

# Create color normalization
vmin = np.min(u)
vmax = np.max(u)
from matplotlib.colors import Normalize
norm = Normalize(vmin=vmin, vmax=vmax)

# Create plots with vertical arrangement
fig, axs = plt.subplots(3, 1, sharex=True, sharey=True,constrained_layout=True)

# Set axis limits
for ax in axs:
    ax.set_xlim(0, 3)
    ax.set_ylim(0, 1)
    ax.set_aspect('equal')

# Create triangulation object
triang = tri.Triangulation(t, x)

# Plot tricontour 1
cn1 = axs[0].tricontourf(triang, u, cmap='viridis', norm=norm)
axs[0].set_title('real')

# Plot tricontour 2
cn2 = axs[1].tricontourf(triang, plot_output[:, 0], cmap='viridis', norm=norm)
axs[1].set_title('pred')

# Plot tricontour 3
cn3 = axs[2].tricontourf(triang, plot_output[:, 0] - u, cmap='viridis', norm=norm)
axs[2].set_title('error')


cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
cbar = fig.colorbar(cn1, cax=cbar_ax, orientation='vertical')



ModuleNotFoundError: No module named 'user_fun'